In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from Style_agmenting.styleaug import StyleAugmentor

import torch
from torchvision.transforms import ToTensor, ToPILImage
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import numpy as np

import easydict 
import json
import argparse
import os
import numpy as np
import cv2
from tqdm import tqdm

# PyTorch Tensor <-> PIL Image transforms:
toTensor = ToTensor()
toPIL = ToPILImage()

def main():

    # Read labels from JSON file
    jsonfile = 'train.json'
    print(f'Reading JSON file from {jsonfile}...')
    with open(jsonfile, 'r') as f:
        labels = json.load(f) # list
    for idx in tqdm(range(len(labels))):

        # ---------- Read image & resize & save
        filename = labels[idx]['filename']
        image    = cv2.imread(os.path.join('Datasets/synthetic/images', filename), cv2.IMREAD_COLOR)
        im_torch = toTensor(image).unsqueeze(0) # 1 x 3 x 256 x 256
        im_torch = im_torch.to('cuda:0' if torch.cuda.is_available() else 'cpu')

        # create style augmentor:
        augmentor = StyleAugmentor()

        # randomize style:
        im_restyled = augmentor(im_torch)
        image = toPIL(im_restyled.squeeze().cpu())
        image = np.array(image)
        
        image    = cv2.resize(image,(768,512))
        cv2.imwrite(os.path.join('Datasets/synthetic/styles_768x512_RGB', filename), image)

    print('done\n\n') 



if __name__=='__main__':
      main()